In [ ]:
import os
import pandas as pd

neo4j_config = {
    "password": "password"
}

In [2]:
from copy import deepcopy
import json
import pprint
pp = pprint.PrettyPrinter(depth=6)

from py2neo import Graph
neo4j_con = Graph(**neo4j_config)

In [3]:
def get_incoming_outgoing_prods(product_id):
    query_incoming_prods = """MATCH (p1:Product)-[r:BOUGHT_WITH]->(p2:Product) WHERE p2.product_id={product_id} 
                                AND p1.group=p2.group RETURN p1 ORDER BY p1.avg_rating DESC, toInteger(p1.salesrank)
                                LIMIT 50"""
    
    query_outgoing_prods = """MATCH (p1:Product)-[r:BOUGHT_WITH]->(p2:Product) WHERE p1.product_id={product_id} 
                                AND p1.group=p2.group RETURN p2 ORDER BY p2.avg_rating DESC, toInteger(p1.salesrank) 
                                LIMIT 50"""
    
    # Fetching incoming nodes TO p1. e.g. (p5, p6, p7) -> p1
    # Belonging to same group(category) and sorted on the basis of `avg_rating` & 'salesrank'
    cur = neo4j_con.run(query_incoming_prods, {"product_id": product_id})
    incoming_prods_nodes = []
    for node_tups in cur:
        for node_tup in node_tups.values():
            nodes_dict = {k: v for k, v in node_tup.iteritems()}
            incoming_prods_nodes.append(nodes_dict)
    
    # Fetching outgoing nodes FROM p1. e.g. p1 -> (p2, p3)
    # Belonging to same group(category) and sorted on the basis of `avg_rating` & 'salesrank'
    cur = neo4j_con.run(query_outgoing_prods, {"product_id": product_id})
    outgoing_prods_nodes = []
    for node_tups in cur:
        for node_tup in node_tups.values():
            nodes_dict = {k: v for k, v in node_tup.iteritems()}
            outgoing_prods_nodes.append(nodes_dict)
    
    return incoming_prods_nodes, outgoing_prods_nodes

In [4]:
def get_recommendations(product_id):

    # Recommendations for product(product_id = 70)
    incoming_prods_nodes, outgoing_prods_nodes = get_incoming_outgoing_prods(product_id)

    # Merging incoming & outgoing nodes. e.g. (p5, p6, p7) + (p2, p3)
    incoming_prods_nodes.extend(outgoing_prods_nodes)
    # print(incoming_prods_nodes)

    # Sorting the nodes on the basis of `avg_rating`.
    #incoming_prods_nodes.sort(key=lambda prod: (-prod['avg_rating'], int(prod['salesrank'])))
    
    recommended_prod_ids = set(map(lambda prod: prod["product_id"], incoming_prods_nodes))
    return recommended_prod_ids

In [5]:
def get_prods_details(product_ids):
    q = """MATCH (p1:Product) WHERE p1.product_id IN {product_ids} return p1.product_id as `Product Id`,
            p1.title as Title, p1.avg_rating as `Avg Rating`, p1.salesrank as `Sales Rank`, p1.group as Group,
            p1.reviews_total as `Reviews Count`
        """
    prods = neo4j_con.data(q, {'product_ids': product_ids})
    return prods

In [6]:
product_id = input("Enter product_id for which you want to get recommendations - ")
prods = get_prods_details([product_id])
#pp.pprint(prods)
input_prod_df = pd.DataFrame(data=prods)
print("\n******* Input product is *******")
input_prod_df

Enter product_id for which you want to get recommendations - 10

******* Input product is *******


,Avg Rating,Group,Product Id,Reviews Count,Sales Rank,Title
0,4.0,Book,10,6,220379,The Edward Said Reader


In [7]:
print("\n******* Recommendations for product(product_i={}) are ******* ".format(product_id))
recommended_prod_ids = list(get_recommendations(product_id))
recommended_prods = get_prods_details(recommended_prod_ids)
recommended_prods_df = pd.DataFrame(data=recommended_prods)
recommended_prods_df.sort_values(['Avg Rating', 'Sales Rank'], ascending=[0, 1], inplace=True)
recommended_prods_df


******* Recommendations for product(product_i=10) are ******* 


,Avg Rating,Group,Product Id,Reviews Count,Sales Rank,Title
13,5.0,Book,111931,2,1005316,Grow! Babies!
9,5.0,Book,88582,1,1207268,Tentacles! : Tales of the Giant Squid (Step In...
8,5.0,Book,137,4,129355,Apples of Gold: A Six-Week Nurturing Program f...
23,5.0,Book,14289,1,1439998,"A Prayer, A Story, A Guide"
3,5.0,Book,11699,6,552678,Getting to the Better Future: A Matter of Cons...
24,4.5,Book,11700,80,1216188,Middlemarch (Wordsworth Collection)
12,4.5,Book,107225,2,1485091,"African Angels : Poetry for the Mind, Body and..."
7,4.5,Book,37570,2,186079,ActionScript for Flash MX Pocket Reference (Po...
11,4.5,Book,29922,2,220454,Plants of Arizona
6,4.5,Book,245484,5,280178,Java Management Extensions
